In [4]:
!pip install rarfile

import rarfile
import os

# Specify the path to your rar file and the extraction directory
rar_path = "/content/deepfake_sample_dataset.rar"  # Note: I assume the rar file is at /content/deepfake_sample_dataset.rar
extract_path = "/content/dataset_rar"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Open and extract the rar file
try:
    with rarfile.RarFile(rar_path, 'r') as rf:
        rf.extractall(extract_path)
    print(f"Successfully extracted {rar_path} to {extract_path}")
except rarfile.Error as e:
    print(f"Error extracting rar file: {e}")
except FileNotFoundError:
    print(f"Error: The file {rar_path} was not found.")

# Check if it worked
if os.path.exists(extract_path):
    print("Extracted contents:")
    print(os.listdir(extract_path))
else:
    print("Extraction path does not exist.")

Successfully extracted /content/deepfake_sample_dataset.rar to /content/dataset_rar
Extracted contents:
['deepfake_sample_dataset']


In [11]:
!pip install gradio --quiet


In [6]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# 1. Set up data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 2. Load the dataset
# Update the data_dir to point to the extracted dataset
data_dir = "/content/dataset_rar/deepfake_sample_dataset"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# 3. Setup the model (Transfer Learning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)

# Replace final layer for 3 classes (real, ai, drawn)
model.fc = nn.Linear(model.fc.in_features, 3)
model.to(device)

# 4. Train the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. Training loop (small demo for 5 epochs)
for epoch in range(5):
    running_loss = 0.0
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")

# 6. Save the trained model
torch.save(model.state_dict(), "/content/deepfake_model.pth")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 77.7MB/s]


Epoch 1, Loss: 1.2580
Epoch 2, Loss: 0.0142
Epoch 3, Loss: 0.0019
Epoch 4, Loss: 0.0014
Epoch 5, Loss: 0.0008


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Using test image: /content/dataset_rar/deepfake_sample_dataset/real/image_2.jpg
The predicted class for the image is: real


In [12]:
import gradio as gr

# Ensure model is in eval mode
model.eval()

def classify_image(img):
    from PIL import Image
    import torch
    import torchvision.transforms as transforms

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    img = img.convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        pred = torch.argmax(output, dim=1).item()

    return dataset.classes[pred]


In [13]:
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs="label",
    title="🔍 DeepFake Detection",
    description="Upload an image to detect if it's REAL, AI-generated, or HAND-DRAWN.",
    examples=[
        ["/content/dataset/real/sample1.jpg"],  # Replace with real paths
        ["/content/dataset/ai/sample2.jpg"],
        ["/content/dataset/drawn/sample3.jpg"]
    ]
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://48aa4e64202ff2ce12.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [15]:
import gradio as gr
from PIL import Image
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn

# Load the saved model (assuming it's already loaded in the environment)
# If not, you would need to add code here to load it.
# Example:
# model_path = "/content/deepfake_model.pth"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = models.resnet18(pretrained=False)
# model.fc = nn.Linear(model.fc.in_features, 3)
# model.load_state_dict(torch.load(model_path, map_location=device))
# model.to(device)
model.eval() # Ensure model is in eval mode

def classify_image(img: Image.Image) -> str:
    """
    Classifies an uploaded image as real, AI-generated, or drawn and returns a descriptive paragraph.

    Args:
        img: The uploaded image as a PIL Image object.

    Returns:
        A string containing a short paragraph describing the classification.
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    img = img.convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        pred = torch.argmax(output, dim=1).item()

    # Assuming the order is the same as during training
    class_labels = ['ai', 'drawn', 'real']
    predicted_class = class_labels[pred]

    # Generate the descriptive paragraph based on the prediction
    if predicted_class == 'real':
        return "Based on the analysis, this image appears to be a real photograph captured by a camera."
    elif predicted_class == 'ai':
        return "Based on the analysis, this image is likely AI-generated. It exhibits characteristics often found in images created by artificial intelligence models."
    elif predicted_class == 'drawn':
        return "Based on the analysis, this image seems to be hand-drawn or created using drawing tools. It displays traits typical of artistic creations."
    else:
        return "Unable to classify the image."


interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs="text",  # Change output type to "text" for paragraph output
    title="🔍 DeepFake Detection",
    description="Upload an image to detect if it's REAL, AI-generated, or HAND-DRAWN and get a descriptive analysis.",
    examples=[
        ["/content/dataset_rar/deepfake_sample_dataset/real/image_1.jpg"],
        ["/content/dataset_rar/deepfake_sample_dataset/ai/image_1.jpg"],
        ["/content/dataset_rar/deepfake_sample_dataset/drawn/image_1.jpg"]
    ]
)

interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://66b068deb4a4a7172a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
import os

# Define the project directory
project_dir = "/content/deepfake_detection_project"
os.makedirs(project_dir, exist_ok=True)

# Create a directory for the model
model_dir = os.path.join(project_dir, "model")
os.makedirs(model_dir, exist_ok=True)

# Create a directory for the dataset (optional, if you want to include a sample)
# dataset_dir = os.path.join(project_dir, "dataset")
# os.makedirs(dataset_dir, exist_ok=True)

# Save the trained model file
# Assuming the model was saved at /content/deepfake_model.pth
model_path_colab = "/content/deepfake_model.pth"
model_path_project = os.path.join(model_dir, "deepfake_model.pth")
if os.path.exists(model_path_colab):
    with open(model_path_colab, 'rb') as f_in, open(model_path_project, 'wb') as f_out:
        f_out.write(f_in.read())
    print(f"Model file saved to {model_path_project}")
else:
    print(f"Model file not found at {model_path_colab}. Skipping model save.")


# Create the Python file for the Gradio app
gradio_app_code = """
import gradio as gr
from PIL import Image
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import os

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model
# Assuming the model file is in the 'model' subfolder
model_path = "model/deepfake_model.pth"

# Load the same model architecture used for training
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 3)

# Load the state dict, handling potential missing keys if architecture differs slightly
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
except RuntimeError as e:
    print(f"Error loading model state_dict: {e}")
    print("Attempting to load with strict=False...")
    model.load_state_dict(torch.load(model_path, map_location=device), strict=False)


model.to(device)
model.eval() # Set the model to evaluation mode

# Define the same transforms used for training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def classify_image(img: Image.Image) -> str:
    \"\"\"
    Classifies an uploaded image as real, AI-generated, or drawn and returns a descriptive paragraph.

    Args:
        img: The uploaded image as a PIL Image object.

    Returns:
        A string containing a short paragraph describing the classification.
    \"\"\"
    img = img.convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        pred = torch.argmax(output, dim=1).item()

    # Assuming the order is the same as during training
    # It's best to get the class labels from the dataset if possible,
    # but for this example, we'll use the assumed order.
    class_labels = ['ai', 'drawn', 'real']
    predicted_class = class_labels[pred]

    # Generate the descriptive paragraph based on the prediction
    if predicted_class == 'real':
        return "Based on the analysis, this image appears to be a real photograph captured by a camera."
    elif predicted_class == 'ai':
        return "Based on the analysis, this image is likely AI-generated. It exhibits characteristics often found in images created by artificial intelligence models."
    elif predicted_class == 'drawn':
        return "Based on the analysis, this image seems to be hand-drawn or created using drawing tools. It displays traits typical of artistic creations."
    else:
        return "Unable to classify the image."

# Example usage (optional, could be removed for the final app.py)
# if __name__ == "__main__":
#     # Example of how to use the function with a local image file
#     # Make sure to have a test image available in your project structure
#     test_image_path = "path/to/your/test_image.jpg" # Replace with a valid path
#     if os.path.exists(test_image_path):
#         prediction = classify_image(Image.open(test_image_path))
#         print(f"Prediction for {test_image_path}: {prediction}")
#     else:
#         print(f"Test image not found at {test_image_path}")


interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="🔍 DeepFake Detection",
    description="Upload an image to detect if it's REAL, AI-generated, or HAND-DRAWN and get a descriptive analysis.",
    # Examples would need to be paths relative to where the app is run,
    # or you would need to include sample images in your project structure.
    # For simplicity in deployment, examples might be omitted or handled differently.
    # examples=[
    #     ["path/to/sample_real.jpg"],
    #     ["path/to/sample_ai.jpg"],
    #     ["path/to/sample_drawn.jpg"]
    # ]
)

if __name__ == "__main__":
    interface.launch()

"""

gradio_app_file = os.path.join(project_dir, "app.py")
with open(gradio_app_file, "w") as f:
    f.write(gradio_app_code)
print(f"Gradio app code saved to {gradio_app_file}")

# Create a requirements.txt file
requirements_content = """
torch
torchvision
gradio
Pillow
rarfile # If you need to handle rar files in your local environment
"""

requirements_file = os.path.join(project_dir, "requirements.txt")
with open(requirements_file, "w") as f:
    f.write(requirements_content)
print(f"Requirements file saved to {requirements_file}")

print("\nProject structure created. You can download the folder:")
print(project_dir)
print("To run the Gradio app locally:")
print(f"1. Navigate to the project directory: cd {project_dir}")
print("2. Install dependencies: pip install -r requirements.txt")
print("3. Run the app: python app.py")

Model file saved to /content/deepfake_detection_project/model/deepfake_model.pth
Gradio app code saved to /content/deepfake_detection_project/app.py
Requirements file saved to /content/deepfake_detection_project/requirements.txt

Project structure created. You can download the folder:
/content/deepfake_detection_project
To run the Gradio app locally:
1. Navigate to the project directory: cd /content/deepfake_detection_project
2. Install dependencies: pip install -r requirements.txt
3. Run the app: python app.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')